In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
rdat = pd.read_csv('RegularSeasonCompactResults.csv')
tdat = pd.read_csv('TourneyCompactResults.csv')
dat = pd.concat([rdat, tdat])

In [5]:
dat.shape

(147339, 8)

In [6]:
dat.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [20]:
games = dat.loc[:, ['Wteam', 'Lteam']].copy()

In [21]:
index2code = pd.Series(sorted(games.Wteam.unique()))

In [22]:
index2code.head()

0    1101
1    1102
2    1103
3    1104
4    1105
dtype: int64

In [23]:
code2index = index2code.reset_index().set_index(0).loc[:, 'index']

In [24]:
code2index.head()

0
1101    0
1102    1
1103    2
1104    3
1105    4
Name: index, dtype: int64

In [25]:
games.loc[:, 'Windex'] = games.Wteam.map(code2index)
games.loc[:, 'Lindex'] = games.Lteam.map(code2index)

In [26]:
games.head()

,Wteam,Lteam,Windex,Lindex
0,1228,1328,127,227
1,1106,1354,5,253
2,1112,1223,11,122
3,1165,1432,64,331
4,1192,1447,91,346


In [27]:
games.loc[:, 'Min_index'] = games.loc[:, ['Windex', 'Lindex']].min(axis=1)
games.loc[:, 'Max_index'] = games.loc[:, ['Windex', 'Lindex']].max(axis=1)

In [28]:
games.head()

,Wteam,Lteam,Windex,Lindex,Min_index,Max_index
0,1228,1328,127,227,127,227
1,1106,1354,5,253,5,253
2,1112,1223,11,122,11,122
3,1165,1432,64,331,64,331
4,1192,1447,91,346,91,346


In [29]:
games.loc[:, 'game'] = games.Min_index.apply(str) + ', ' + games.Max_index.apply(str)

In [30]:
games.head()

,Wteam,Lteam,Windex,Lindex,Min_index,Max_index,game
0,1228,1328,127,227,127,227,"127, 227"
1,1106,1354,5,253,5,253,"5, 253"
2,1112,1223,11,122,11,122,"11, 122"
3,1165,1432,64,331,64,331,"64, 331"
4,1192,1447,91,346,91,346,"91, 346"


In [46]:
games.loc[:, 'win_lose'] = games.Windex.apply(str) + ', ' + games.Lindex.apply(str)

In [47]:
gpd_win = games.groupby(['game', 'win_lose'])
gpd_game = games.groupby('game')

In [48]:
game_counts = gpd_game.size()
win_lose_history = gpd_win.size()

In [50]:
game_counts.head()

game
0, 112    1
0, 121    1
0, 122    5
0, 129    6
0, 133    1
dtype: int64

In [51]:
win_lose_history.head()

game    win_lose
0, 112  112, 0      1
0, 121  121, 0      1
0, 122  0, 122      3
        122, 0      2
0, 129  129, 0      6
dtype: int64

In [61]:
min_ncontests = 20

win_rate = win_lose_history.div(game_counts.loc[game_counts >= min_ncontests], level=0).dropna()

In [70]:
win_rate.head(20)

game     win_lose
1, 117   1, 117      0.379310
         117, 1      0.620690
1, 206   1, 206      0.253968
         206, 1      0.746032
1, 260   1, 260      0.318182
         260, 1      0.681818
1, 323   1, 323      0.255814
         323, 1      0.744186
1, 327   1, 327      0.207547
         327, 1      0.792453
1, 330   1, 330      0.129032
         330, 1      0.870968
1, 360   1, 360      0.323529
         360, 1      0.676471
1, 39    1, 39       0.181818
         39, 1       0.818182
1, 60    1, 60       0.294118
         60, 1       0.705882
10, 101  10, 101     0.509434
         101, 10     0.490566
dtype: float64

In [71]:
always_win = win_rate.loc[win_rate >= 1]

always_win.name = 'percentage'

always_win = always_win.reset_index()

always_win

In [82]:
always_win.loc[:, 'Wteam_index'] = always_win.win_lose.str.extract(r'(\d+), \d+', expand=False)
always_win.loc[:, 'Lteam_index'] = always_win.win_lose.str.extract(r'\d+, (\d+)', expand=False)

In [83]:
always_win

,game,win_lose,percentage,Wteam_index,Lteam_index
0,"11, 218","11, 218",1.0,11,218
1,"160, 169","160, 169",1.0,160,169
2,"237, 251","237, 251",1.0,237,251
3,"54, 101","54, 101",1.0,54,101
4,"58, 292","292, 58",1.0,292,58
5,"64, 292","292, 64",1.0,292,64
6,"71, 80","80, 71",1.0,80,71


In [91]:
always_win.loc[:, 'Wteam_code'] = always_win.loc[:, 'Wteam_index'].astype('int').map(index2code)
always_win.loc[:, 'Lteam_code'] = always_win.loc[:, 'Lteam_index'].astype('int').map(index2code)

In [92]:
always_win

,game,win_lose,percentage,Wteam_index,Lteam_index,Wteam_code,Lteam_code
0,"11, 218","11, 218",1.0,11,218,1112,1319
1,"160, 169","160, 169",1.0,160,169,1261,1270
2,"237, 251","237, 251",1.0,237,251,1338,1352
3,"54, 101","54, 101",1.0,54,101,1155,1202
4,"58, 292","292, 58",1.0,292,58,1393,1159
5,"64, 292","292, 64",1.0,292,64,1393,1165
6,"71, 80","80, 71",1.0,80,71,1181,1172


In [95]:
teams = pd.read_csv('Teams.csv').set_index('Team_Id')
teams.head()

In [99]:
always_win.loc[:, 'Wteam'] = always_win.Wteam_code.map(teams.Team_Name)
always_win.loc[:, 'Lteam'] = always_win.Lteam_code.map(teams.Team_Name)

In [100]:
always_win

,game,win_lose,percentage,Wteam_index,Lteam_index,Wteam_code,Lteam_code,Wteam,Lteam
0,"11, 218","11, 218",1.0,11,218,1112,1319,Arizona,Northern Arizona
1,"160, 169","160, 169",1.0,160,169,1261,1270,LSU,McNeese St
2,"237, 251","237, 251",1.0,237,251,1338,1352,Pittsburgh,Robert Morris
3,"54, 101","54, 101",1.0,54,101,1155,1202,Clemson,Furman
4,"58, 292","292, 58",1.0,292,58,1393,1159,Syracuse,Colgate
5,"64, 292","292, 64",1.0,292,64,1393,1165,Syracuse,Cornell
6,"71, 80","80, 71",1.0,80,71,1181,1172,Duke,Davidson
